In [6]:
import pandas as pd
import numpy as np
import json
import calculations
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import statsmodels.api as sm
import warnings
from scipy.stats import iqr, skew, kurtosis, pearsonr

In [26]:
# Disable runtime warnings due to datasets containing NaNs after skew/kurtosis calcs
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Process data

stimuli = []
scale_type = {
    "gesture_pitch_number": 100,
    "gesture_pitch_greyscale": 49,
    "gesture_roll_number": 100,
    "gesture_roll_greyscale": 49,
    "slider_number": 1,
    "slider_greyscale": 1,
}

with open('experiment_data/combined_participant_data.json', 'r') as file:
    data = json.load(file)

    if data:
        for participant in data:
            participant_id = participant.get("id")
            for experiment in participant["completedExperiments"]:
                experiment_type = experiment.get("experimentType")
                started_date = experiment.get("startedDate")
                ended_date = experiment.get("endedDate")
                for stimulus in experiment["successfulStimuli"]:
                    sensor_reading = stimulus.get("sensorReading", {})
                    quaternion = (
                        sensor_reading.get("w"),
                        sensor_reading.get("x"),
                        sensor_reading.get("y"),
                        sensor_reading.get("z")
                    )
                    # Remap quaternion -- the stored data from device app is in an incorrect order (actual values of w, x, y, z -> x, y, z, w)
                    x, y, z, w = quaternion
                    _, roll, pitch = calculations.quaternion_to_euler(x, y, z, w)
                    
                    sensor_reading["roll"] = roll
                    sensor_reading["pitch"] = pitch

                    pitch_scaled = calculations.scale_pitch(pitch, scale_type[experiment_type])
                    roll_scaled = calculations.scale_roll(roll, scale_type[experiment_type])
                    truth_value = stimulus.get("truth")
                    slider_value = stimulus.get("value")
                    
                    pitch_difference = abs(truth_value - pitch_scaled)
                    roll_difference = abs(truth_value - roll_scaled)

                    stimulus["participant_id"] = participant_id
                    stimulus["pitch"] = pitch
                    stimulus["pitch_truth"] = pitch_scaled
                    stimulus["pitch_truth_diff"] = pitch_difference
                
                    stimulus["roll"] = roll
                    stimulus["roll_truth"] = roll_scaled
                    stimulus["roll_truth_diff"] = roll_difference
                    
                    stimulus["slider_diff"] = abs(truth_value - slider_value)
                    
                    if truth_value == 0:
                        stimulus["pitch_truth_diff_percentage"] = (pitch_scaled / scale_type[experiment_type]) * 100
                        stimulus["roll_truth_diff_percentage"] = (roll_scaled / scale_type[experiment_type]) * 100
                    else:
                        stimulus["pitch_truth_diff_percentage"] = (pitch_difference / truth_value) * 100
                        stimulus["roll_truth_diff_percentage"] = (roll_difference / truth_value) * 100
                        
                    stimulus["experiment_type"] = experiment_type
                    stimulus["experiment_time_spent"] = ended_date - started_date
                    stimulus["calibration"] = sensor_reading.get("calibration_status")
                            
                    stimuli.append(stimulus)

df = pd.DataFrame(data=stimuli)

In [36]:
display(df)

,value,id,truth,inputType,sensorReading,participant_id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
0,50.782763,1,49,slider,"{'stability': 'In motion', 'activity': 'Still'...",1,-8.758405,0,49,-6.629249,0,49,1.782763,100.000000,100.000000,slider_number,-183.397782,3
1,21.909768,2,32,slider,"{'y': 0.411926, 'activity_confidence': 92, 'x'...",1,-8.758405,0,32,-6.629249,0,32,10.090232,100.000000,100.000000,slider_number,-183.397782,3
2,74.312516,3,76,slider,"{'y': 0.411926, 'timestamp': 1719216751, 'dura...",1,-8.758405,0,76,-6.629249,0,76,1.687484,100.000000,100.000000,slider_number,-183.397782,3
3,1.492341,4,6,slider,"{'stability': 'In motion', 'z': 0.906189, 'cal...",1,-8.758405,0,6,-6.629249,0,6,4.507659,100.000000,100.000000,slider_number,-183.397782,3
4,9.036517,5,26,slider,"{'y': 0.411926, 'timestamp': 1719216751, 'cali...",1,-8.758405,0,26,-6.629249,0,26,16.963483,100.000000,100.000000,slider_number,-183.397782,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,0.000000,16,0,device,"{'stability': 'Stable', 'z': 0.442078, 'w': 0....",33,49.661682,27,27,3.824130,2,2,0.000000,55.102041,4.081633,gesture_pitch_greyscale,-170.829092,2
3956,0.000000,17,47,device,"{'y': 0.879944, 'stability': 'Stable', 'x': 0....",33,2.677436,1,46,-1.854517,1,46,47.000000,97.872340,97.872340,gesture_pitch_greyscale,-170.829092,2
3957,0.000000,18,3,device,"{'duration': 210, 'activity_confidence': 97, '...",33,76.534100,42,39,4.726522,3,0,3.000000,1300.000000,0.000000,gesture_pitch_greyscale,-170.829092,2
3958,0.000000,19,26,device,"{'w': -0.00836182, 'timestamp': 1719572804, 'y...",33,1.448705,1,25,1.770745,1,25,26.000000,96.153846,96.153846,gesture_pitch_greyscale,-170.829092,2


In [8]:
truth_value_batches_100 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)],
    [i for i in range(50, 60)],
    [i for i in range(60, 70)],
    [i for i in range(70, 80)],
    [i for i in range(80, 90)],
    [i for i in range(90, 101)]
]

truth_value_batches_49 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)]
]

experiment_truth_type = {
    "gesture_pitch_number": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_pitch_greyscale": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_roll_number": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "gesture_roll_greyscale": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "slider_number": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
    "slider_greyscale": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
}

In [9]:
calibration_threshold = 2
g_pitch_greyscale_df = df[(df["experiment_type"] == "gesture_pitch_greyscale") & (df["calibration"] >= calibration_threshold)]
g_pitch_number_df = df[(df["experiment_type"] == "gesture_pitch_number") & (df["calibration"] >= calibration_threshold)]
g_roll_greyscale_df = df[(df["experiment_type"] == "gesture_roll_greyscale") & (df["calibration"] >= calibration_threshold)]
g_roll_number_df = df[(df["experiment_type"] == "gesture_roll_number") & (df["calibration"] >= calibration_threshold)]
g_slider_greyscale_df = df[(df["experiment_type"] == "slider_greyscale")]
g_slider_number_df = df[(df["experiment_type"] == "slider_number")]

fig = sp.make_subplots(rows=6, cols=2, subplot_titles=(
    'Pitch - Greyscale (Scaled 0-49)',
    'Pitch Error - Greyscale (Scaled 0-49)',
    'Pitch - Number (Scaled 0-100)',
    'Pitch Error - Number (Scaled 0-100)',
    'Roll - Greyscale (Scaled 0-49)',
    'Roll Error - Greyscale (Scaled 0-49)',
    'Roll - Number (Scaled 0-100)',
    'Roll Error - Number (Scaled 0-100)',
    'Slider - Greyscale (Scaled 0-49)',
    'Slider Error - Greyscale (Scaled 0-49)',
    'Slider - Number (Scaled 0-100)',
    'Slider Error - Number (Scaled 0-100)'
))

fig.add_trace(px.scatter(g_pitch_greyscale_df, x="truth", y="pitch_truth", color_discrete_sequence=['blue']).data[0], row=1, col=1)
fig.add_trace(px.scatter(g_pitch_greyscale_df, x="truth", y="pitch_truth", error_y="pitch_truth_diff", color_discrete_sequence=['blue']).data[0], row=1, col=2)
fig.add_trace(px.scatter(g_pitch_number_df, x="truth", y="pitch_truth", color_discrete_sequence=['green']).data[0], row=2, col=1)
fig.add_trace(px.scatter(g_pitch_number_df, x="truth", y="pitch_truth", error_y="pitch_truth_diff", color_discrete_sequence=['green']).data[0], row=2, col=2)
fig.add_trace(px.scatter(g_roll_greyscale_df, x="truth", y="roll_truth", color_discrete_sequence=['red']).data[0], row=3, col=1)
fig.add_trace(px.scatter(g_roll_greyscale_df, x="truth", y="roll_truth", error_y="roll_truth_diff", color_discrete_sequence=['red']).data[0], row=3, col=2)
fig.add_trace(px.scatter(g_roll_number_df, x="truth", y="roll_truth", color_discrete_sequence=['purple']).data[0], row=4, col=1)
fig.add_trace(px.scatter(g_roll_number_df, x="truth", y="roll_truth", error_y="roll_truth_diff", color_discrete_sequence=['purple']).data[0], row=4, col=2)
fig.add_trace(px.scatter(g_slider_greyscale_df, x="truth", y="value", color_discrete_sequence=['magenta']).data[0], row=5, col=1)
fig.add_trace(px.scatter(g_slider_greyscale_df, x="truth", y="value", error_y="slider_diff", color_discrete_sequence=['magenta']).data[0], row=5, col=2)
fig.add_trace(px.scatter(g_slider_number_df, x="truth", y="value", color_discrete_sequence=['purple']).data[0], row=6, col=1)
fig.add_trace(px.scatter(g_slider_number_df, x="truth", y="value", error_y="slider_diff", color_discrete_sequence=['purple']).data[0], row=6, col=2)

fig.update_layout(height=2000, width=1600, title_text="Sensor Readings Scatter Plots")
fig.update_xaxes(title_text="Truth Values")
fig.update_yaxes(title_text="Sensor Readings")

fig['layout']['yaxis1'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis2'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis3'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis4'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis5'].update(title='Roll Sensor Readings')
fig['layout']['yaxis6'].update(title='Roll Sensor Readings')
fig['layout']['yaxis7'].update(title='Roll Sensor Readings')
fig['layout']['yaxis8'].update(title='Roll Sensor Readings')
fig['layout']['yaxis9'].update(title='Slider Readings')
fig['layout']['yaxis10'].update(title='Slider Readings')
fig['layout']['yaxis11'].update(title='Slider Readings')
fig['layout']['yaxis12'].update(title='Slider Readings')

fig.show()

In [10]:
def create_deviation_plots(truth_values, experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['truth'].isin(truth_values)) & 
                     (df['inputType'] == truth_type["input_type"])]

    mean_truth = filtered_df.groupby("truth")[truth_type["truth"]].mean().reset_index()
    std_truth = filtered_df.groupby("truth")[truth_type["truth"]].std().reset_index()

    mean_std_df = pd.merge(mean_truth, std_truth, on="truth", suffixes=('_mean', '_std'))
    
    # Have to fill NaNs with zeroes, needed if one or more truths only have one reading
    mean_std_df[f'{truth_type["truth"]}_std'] = mean_std_df[f'{truth_type["truth"]}_std'].fillna(0)
    mean_std_df[f'{truth_type["truth"]}_std'] = mean_std_df[f'{truth_type["truth"]}_std'].abs()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=mean_std_df["truth"], y=mean_std_df[f'{truth_type["truth"]}_mean'],
        mode='lines', line=dict(color='blue'), name=f'Mean {truth_type["label"]}'
    ))

    upper_bound = mean_std_df[f'{truth_type["truth"]}_mean'] + mean_std_df[f'{truth_type["truth"]}_std']
    lower_bound = mean_std_df[f'{truth_type["truth"]}_mean'] - mean_std_df[f'{truth_type["truth"]}_std']
    
    fig.add_trace(go.Scatter(
        x=mean_std_df["truth"], y=upper_bound,
        fill=None, mode='lines', line=dict(color='lightblue'), name='Upper Deviation Range', showlegend=False
    ))
    fig.add_trace(go.Scatter(
        x=mean_std_df["truth"], y=lower_bound,
        fill='tonexty', mode='lines', line=dict(color='lightblue'), name='Deviation Range'
    ))

    fig.update_layout(
        title=f"Deviation Plot for {truth_type['label']} - {experiment_type}",
        xaxis_title="Truth Values",
        yaxis_title=f"{truth_type['label']} Values",
        hovermode="x"
    )

    fig.show()
    
create_deviation_plots([i for i in range(0, 101)], "gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
    
for batch in truth_value_batches_100:
    create_deviation_plots(batch, "gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
    
create_deviation_plots([i for i in range(0, 50)], "gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])

for batch in truth_value_batches_49:
    create_deviation_plots(batch, "gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])
    
create_deviation_plots([i for i in range(0, 101)], "gesture_roll_number", experiment_truth_type["gesture_roll_number"])

for batch in truth_value_batches_100:
    create_deviation_plots(batch, "gesture_roll_number", experiment_truth_type["gesture_roll_number"])

create_deviation_plots([i for i in range(0, 50)], "gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])

for batch in truth_value_batches_49:
    create_deviation_plots(batch, "gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])
 
create_deviation_plots([i for i in range(0, 101)], "slider_number", experiment_truth_type["slider_number"])
    
for batch in truth_value_batches_100:
    create_deviation_plots(batch, "slider_number", experiment_truth_type["slider_number"])

create_deviation_plots([i for i in range(0, 50)], "slider_greyscale", experiment_truth_type["slider_greyscale"])

for batch in truth_value_batches_49:
    create_deviation_plots(batch, "slider_greyscale", experiment_truth_type["slider_greyscale"])

In [11]:
def create_histogram_truth_plots(experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['inputType'] == truth_type["input_type"])]

    fig = px.histogram(filtered_df, x=truth_type["truth_diff"],
                    title=f'{truth_type['label']} Differences - {experiment_type}',
                    labels={f'{truth_type["truth_diff"]}': f'{truth_type["label"]} Difference'})
    
    fig.show()
    
for experiment_type, truth_type in experiment_truth_type.items():
    create_histogram_truth_plots(experiment_type, truth_type)

In [12]:
def create_histogram_plots(truth_values, experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['truth'].isin(truth_values)) & 
                     (df['inputType'] == truth_type["input_type"])]

    filtered_df = filtered_df.sort_values(by='truth')

    fig = px.histogram(filtered_df, x=truth_type["truth"], color="truth", 
                       facet_col="truth", marginal="rug", 
                       labels={f'{truth_type["truth"]}': f'{truth_type["label"]}', 'truth': 'Truth'},
                       title=f'Distribution of {truth_type["label"]} Values for Truth Values {truth_values[0]} to {truth_values[-1]} - {experiment_type}',
                       nbins=int(filtered_df[truth_type["truth"]].max() - filtered_df[truth_type["truth"]].min() + 1))

    fig.show()
    
for batch in truth_value_batches_100:
    create_histogram_plots(batch, "gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
    
for batch in truth_value_batches_49:
    create_histogram_plots(batch, "gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])
    
for batch in truth_value_batches_100:
    create_histogram_plots(batch, "gesture_roll_number", experiment_truth_type["gesture_roll_number"])
    
for batch in truth_value_batches_49:
    create_histogram_plots(batch, "gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])

In [44]:
def calculate_summary_statistics(experiment_type, truth_type):
    filtered_df = df[df['experiment_type'] == experiment_type]
    grouped = filtered_df.groupby('truth')[truth_type].agg(['mean', 'std', 'min', 'max', 'count', 'median'])
    grouped['iqr'] = filtered_df.groupby('truth')[truth_type].apply(iqr)
    grouped['skew'] = filtered_df.groupby('truth')[truth_type].apply(skew)
    grouped['kurtosis'] = filtered_df.groupby('truth')[truth_type].apply(kurtosis)
    
    # Kurtosis and skew calculate NaNs on certain truths (0 and 100), but it's not an issue. Filling NaNs with zeroes
    grouped = grouped.fillna(0)
    return grouped.reset_index()

def generate_summary_tables():
    summary_tables = {}
    for experiment_type, truth_info in experiment_truth_type.items():
        summary_table = calculate_summary_statistics(experiment_type, truth_info['truth'])
        summary_table['experiment_type'] = experiment_type
        summary_tables[experiment_type] = summary_table
    return summary_tables

def calculate_truth_diff_percentages(experiment_type, truth_diff_column):
    filtered_df = df[df['experiment_type'] == experiment_type]
    count = filtered_df.shape[0]
    within_5 = (filtered_df[truth_diff_column].abs() <= 5).sum() / count * 100
    within_10 = (filtered_df[truth_diff_column].abs() <= 10).sum() / count * 100
    within_15 = (filtered_df[truth_diff_column].abs() <= 15).sum() / count * 100
    within_20 = (filtered_df[truth_diff_column].abs() <= 20).sum() / count * 100
    within_25 = (filtered_df[truth_diff_column].abs() <= 25).sum() / count * 100
    return within_5, within_10, within_15, within_20, within_25

def calculate_averages_per_experiment_type(summary_tables):
    averages = []
    for experiment_type, summary_table in summary_tables.items():
        within_5, within_10, within_15, within_20, within_25 = calculate_truth_diff_percentages(experiment_type, experiment_truth_type[experiment_type]['truth_diff'])
        
        time_spent_df = df[df['experiment_type'] == experiment_type].drop_duplicates(subset=['participant_id', 'experiment_type'])
        time_spent_avg = time_spent_df['experiment_time_spent'].abs().mean()
        time_spent_min = time_spent_df['experiment_time_spent'].abs().min()
        time_spent_max = time_spent_df['experiment_time_spent'].abs().max()
        
        averages.append({
            'experiment_type': experiment_type,
            'time_spent_avg': time_spent_avg,
            'time_spent_min': time_spent_min,
            'time_spent_max': time_spent_max,
            'mean': summary_table['mean'].mean(),
            'std': summary_table['std'].mean(),
            'min': summary_table['min'].min(),
            'max': summary_table['max'].max(),
            'count': summary_table['count'].count(),
            'median': summary_table['median'].median(),
            'iqr': summary_table['iqr'].mean(),
            'skew': summary_table['skew'].mean(),
            'kurtosis': summary_table['kurtosis'].mean(),
            'within_5_percent': within_5,
            'within_10_percent': within_10,
            'within_15_percent': within_15,
            'within_20_percent': within_20,
            'within_25_percent': within_25
        })
    return pd.DataFrame(averages)

summary_tables = generate_summary_tables()
averages_df = calculate_averages_per_experiment_type(summary_tables)

display(averages_df)

for _, summary_table in summary_tables.items():
    display(summary_table)

,experiment_type,time_spent_avg,time_spent_min,time_spent_max,mean,std,min,max,count,median,iqr,skew,kurtosis,within_5_percent,within_10_percent,within_15_percent,within_20_percent,within_25_percent
0,gesture_pitch_number,143.881667,73.162797,399.222038,46.357116,13.317517,0.0,100.0,101,46.500000,11.202970,0.372431,-0.585424,37.424242,62.121212,76.515152,86.515152,91.818182
1,gesture_pitch_greyscale,152.724591,100.113338,303.560128,21.337875,15.556473,0.0,49.0,50,18.500000,24.295000,0.275407,-1.076463,19.242424,32.575758,48.030303,60.454545,72.424242
2,gesture_roll_number,152.073224,80.542917,437.243360,51.979259,11.723395,0.0,98.0,101,54.000000,12.155941,-0.153931,-0.723704,27.272727,47.424242,66.818182,79.696970,87.727273
3,gesture_roll_greyscale,181.509191,103.378948,501.023651,24.937564,12.852115,0.0,48.0,50,24.250000,18.420000,-0.018345,-0.923217,19.242424,37.575758,53.636364,65.303030,77.424242
4,slider_number,160.304255,65.096681,313.937267,48.739534,3.925547,0.0,100.0,101,50.108070,3.821792,-0.122070,-0.628053,74.090909,93.181818,97.878788,98.939394,99.393939
5,slider_greyscale,158.260002,93.029034,246.268256,23.130258,17.218755,0.0,49.0,50,23.107655,27.976096,0.107296,-1.184886,19.242424,35.303030,49.090909,59.090909,70.151515


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,4.000000,4.444097,0,15,9,3.0,1.00,1.812662,2.464749,gesture_pitch_number
1,1,14.800000,24.993999,0,59,5,3.0,8.00,1.418837,0.132166,gesture_pitch_number
2,2,5.000000,2.160247,2,7,4,5.5,2.00,-0.687243,-1.000000,gesture_pitch_number
3,3,3.000000,2.449490,1,6,4,2.5,3.50,0.314270,-1.592593,gesture_pitch_number
4,4,7.250000,4.494947,2,14,12,5.5,7.50,0.418334,-1.413246,gesture_pitch_number
...,...,...,...,...,...,...,...,...,...,...,...
96,96,91.000000,7.456541,83,100,6,89.0,12.25,0.351951,-1.542777,gesture_pitch_number
97,97,88.250000,11.324752,73,100,4,90.0,9.75,-0.495999,-1.018157,gesture_pitch_number
98,98,86.888889,9.752493,72,100,9,87.0,14.00,-0.290639,-1.150153,gesture_pitch_number
99,99,84.000000,9.230385,74,96,6,80.5,13.00,0.488085,-1.445745,gesture_pitch_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,23.500000,16.400236,0,49,32,23.0,30.00,0.042640,-1.236708,gesture_pitch_greyscale
1,1,24.941176,14.834380,2,49,17,26.0,24.00,0.055817,-1.335211,gesture_pitch_greyscale
2,2,20.583333,18.802603,0,49,12,17.5,35.25,0.339601,-1.396316,gesture_pitch_greyscale
3,3,25.923077,17.394738,1,49,13,33.0,29.00,-0.324369,-1.472361,gesture_pitch_greyscale
4,4,28.538462,16.993211,0,48,13,29.0,27.00,-0.415636,-1.126411,gesture_pitch_greyscale
5,5,21.307692,15.494002,0,49,13,17.0,19.00,0.392375,-1.048165,gesture_pitch_greyscale
6,6,25.750000,15.854247,6,46,8,24.5,25.75,0.069125,-1.694972,gesture_pitch_greyscale
7,7,25.400000,15.049548,0,45,10,23.0,16.25,-0.207428,-0.870340,gesture_pitch_greyscale
8,8,18.071429,17.054051,1,49,14,13.0,24.75,0.663218,-0.885492,gesture_pitch_greyscale
9,9,16.000000,10.760008,6,34,10,11.5,13.75,0.856855,-0.966206,gesture_pitch_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,14.333333,18.929694,1,36,3,6.0,17.50,0.652012,-1.500000,gesture_roll_number
1,1,14.800000,9.391486,1,25,5,13.0,9.00,-0.406706,-0.999479,gesture_roll_number
2,2,19.000000,7.905694,4,28,9,21.0,10.00,-0.766345,-0.561072,gesture_roll_number
3,3,20.142857,10.007140,6,38,7,20.0,8.50,0.482046,-0.208409,gesture_roll_number
4,4,25.750000,5.560276,22,34,4,23.5,3.75,1.080610,-0.723084,gesture_roll_number
...,...,...,...,...,...,...,...,...,...,...,...
96,96,84.000000,14.899664,47,98,14,90.0,15.00,-1.350412,0.773910,gesture_roll_number
97,97,80.333333,17.048949,54,94,6,88.5,23.00,-0.735559,-1.219921,gesture_roll_number
98,98,71.166667,21.094233,36,94,6,74.0,22.75,-0.634737,-0.674530,gesture_roll_number
99,99,79.500000,20.216330,47,98,6,89.0,23.00,-0.785706,-1.033890,gesture_roll_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,23.454545,14.922908,1,47,33,22.0,24.00,0.104473,-1.269198,gesture_roll_greyscale
1,1,21.363636,11.732627,6,44,11,23.0,11.00,0.394040,-0.462697,gesture_roll_greyscale
2,2,25.538462,14.700427,4,48,13,25.0,26.00,0.117892,-1.239018,gesture_roll_greyscale
3,3,28.000000,11.084094,9,46,15,28.0,14.50,-0.105548,-1.012642,gesture_roll_greyscale
4,4,32.363636,13.865589,1,48,11,31.0,17.50,-0.835666,0.380218,gesture_roll_greyscale
5,5,20.769231,10.647017,2,37,13,18.0,14.00,0.072218,-0.975219,gesture_roll_greyscale
6,6,22.500000,12.281422,4,45,22,23.5,18.50,0.165522,-1.081811,gesture_roll_greyscale
7,7,26.384615,11.665751,10,47,13,24.0,16.00,0.549941,-0.901473,gesture_roll_greyscale
8,8,25.700000,13.548678,4,45,10,29.0,17.50,-0.285557,-1.146736,gesture_roll_greyscale
9,9,25.200000,10.564090,4,46,15,25.0,10.50,-0.088711,-0.018847,gesture_roll_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,0.000000,0.000000,0.000000,0.000000,7,0.000000,0.000000,0.000000,0.000000,slider_number
1,1,0.274649,0.329813,0.000000,0.952432,8,0.172917,0.326624,1.170934,0.187467,slider_number
2,2,0.504782,0.547771,0.062684,1.238188,4,0.359127,0.667964,0.579703,-1.262379,slider_number
3,3,1.543033,1.263062,0.121591,3.960529,7,1.289665,1.212593,0.932448,-0.010380,slider_number
4,4,1.438317,0.383883,0.939227,2.096687,7,1.296690,0.343714,0.598814,-0.613947,slider_number
...,...,...,...,...,...,...,...,...,...,...,...
96,96,97.407413,2.354941,94.162639,99.486481,4,97.990266,2.446783,-0.673280,-1.083310,slider_number
97,97,98.535673,0.705188,97.511674,99.248752,8,98.877689,1.096923,-0.537688,-1.415678,slider_number
98,98,99.307784,0.324006,99.078677,99.536891,2,99.307784,0.229107,0.000000,-2.000000,slider_number
99,99,99.702126,0.286189,99.313720,99.953773,4,99.770505,0.317959,-0.615287,-1.166082,slider_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,28.334861,20.286814,0.000000,49.000000,16,39.535285,37.658561,-0.427579,-1.551340,slider_greyscale
1,1,20.590820,16.844100,0.000000,39.440062,11,26.663515,32.008493,-0.070043,-1.766584,slider_greyscale
2,2,18.115094,15.406683,0.000000,47.922947,13,16.752060,17.550015,0.639371,-0.497131,slider_greyscale
3,3,26.151618,20.299737,1.275259,49.000000,13,34.588501,40.057826,-0.104588,-1.806166,slider_greyscale
4,4,11.989843,15.543100,0.000000,47.502549,18,3.124427,20.255006,1.071043,-0.235792,slider_greyscale
5,5,15.618866,17.461250,0.000000,45.316778,14,9.719251,25.524892,0.761219,-0.989774,slider_greyscale
6,6,29.208802,13.440181,5.369027,49.000000,9,30.175503,14.645308,-0.166405,-0.610516,slider_greyscale
7,7,24.111849,19.028522,0.000000,49.000000,19,20.843229,37.374105,0.068376,-1.584428,slider_greyscale
8,8,14.735931,19.623895,0.000000,49.000000,12,3.638726,27.797733,0.882638,-0.847843,slider_greyscale
9,9,23.256567,19.407185,0.000000,49.000000,17,23.277204,39.193603,0.124750,-1.578305,slider_greyscale


In [14]:
def create_and_analyze_plots(experiment_type, truth_info):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['inputType'] == truth_info['input_type'])]

    if filtered_df.empty:
        return

    filtered_df = filtered_df.sort_values(by=['participant_id', 'id'])

    filtered_df['previous_truth'] = filtered_df.groupby('participant_id')[truth_info['truth']].shift(1)
    filtered_df['truth_prev_diff'] = abs(filtered_df[truth_info['truth']] - filtered_df['previous_truth'])

    filtered_df['deviation'] = abs(filtered_df[truth_info['truth']] - filtered_df['truth'])
    filtered_df['next_deviation'] = filtered_df.groupby('participant_id')['deviation'].shift(-1)

    fig1 = px.scatter(filtered_df, x='previous_truth', y=truth_info['truth'], trendline='lowess',
                      title=f'Learning Bias in {truth_info["label"]} Responses - {experiment_type}', trendline_color_override='black')
    fig1.show()

    fig2 = px.scatter(filtered_df, x='deviation', y='next_deviation', trendline='lowess',
                      title=f'Impact of {truth_info["label"]} Deviation on Next Response - {experiment_type}', trendline_color_override='black')
    fig2.show()

    valid_pairs_prev = filtered_df[['previous_truth', truth_info['truth']]].dropna()
    if not valid_pairs_prev.empty:
        prev_corr, prev_pval = pearsonr(valid_pairs_prev['previous_truth'], valid_pairs_prev[truth_info['truth']], alternative='greater')
        print(f"Correlation between previous and current {truth_info['label']} for {experiment_type}: {prev_corr}, p-value: {prev_pval}")

    valid_pairs_dev = filtered_df[['deviation', 'next_deviation']].dropna()
    if not valid_pairs_dev.empty:
        dev_corr, dev_pval = pearsonr(valid_pairs_dev['deviation'], valid_pairs_dev['next_deviation'], alternative='greater')
        print(f"Correlation between {truth_info['label']} deviation and next response for {experiment_type}: {dev_corr}, p-value: {dev_pval}")

create_and_analyze_plots("gesture_pitch_number", experiment_truth_type["gesture_pitch_number"])
create_and_analyze_plots("gesture_pitch_greyscale", experiment_truth_type["gesture_pitch_greyscale"])
create_and_analyze_plots("gesture_roll_number", experiment_truth_type["gesture_roll_number"])
create_and_analyze_plots("gesture_roll_greyscale", experiment_truth_type["gesture_roll_greyscale"])
create_and_analyze_plots("slider_number", experiment_truth_type["slider_number"])
create_and_analyze_plots("slider_greyscale", experiment_truth_type["slider_greyscale"])

Correlation between previous and current Pitch Truth for gesture_pitch_number: 0.21073638061783692, p-value: 5.0136209351305555e-08
Correlation between Pitch Truth deviation and next response for gesture_pitch_number: 0.4720250057046686, p-value: 2.0730850257352147e-36


Correlation between previous and current Pitch Truth for gesture_pitch_greyscale: 0.20052036768488107, p-value: 2.0688603309747581e-07
Correlation between Pitch Truth deviation and next response for gesture_pitch_greyscale: 0.0705877731308327, p-value: 0.03868180762550523


Correlation between previous and current Roll Truth for gesture_roll_number: 0.14685017593768168, p-value: 0.00011221163926951014
Correlation between Roll Truth deviation and next response for gesture_roll_number: 0.27657446613217207, p-value: 8.97398501691106e-13


Correlation between previous and current Roll Truth for gesture_roll_greyscale: 0.10787689030914666, p-value: 0.00342821882164397
Correlation between Roll Truth deviation and next response for gesture_roll_greyscale: 0.005382589746143588, p-value: 0.4464988997125016


Correlation between previous and current Slider Value for slider_number: -0.05807628328794208, p-value: 0.9268259952099894
Correlation between Slider Value deviation and next response for slider_number: 0.07829989902172355, p-value: 0.02501402165354812


Correlation between previous and current Slider Value for slider_greyscale: 0.036370779392971286, p-value: 0.18162051716879998
Correlation between Slider Value deviation and next response for slider_greyscale: -0.04058016795518805, p-value: 0.8448317957522423
